In [2]:
# !unzip Resume-SK.zip

In [2]:
!pip install docling
!pip install -U sentence-transformers
!pip install rank-bm25
!pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.4/164.4 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.7/42.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 103.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 45.0 MB/s eta 0:00:00
   ━━

In [3]:
import os
import json
from pathlib import Path
from typing import Dict, List, Optional
import hashlib
import statistics

from docling.document_converter import DocumentConverter
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from sentence_transformers import SentenceTransformer
import pickle
import re
from typing import List, Dict, Any
from rank_bm25 import BM25Okapi
from langchain.docstore.document import Document


import os
import pickle
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import openai
from openai import OpenAI

import re
from langchain.docstore.document import Document

from google.colab import userdata
import openai

# Constants
RESUME_PDF_PATH = "Submit your resume or CV (File responses)"
RESUME_MARKDOWN_PATH = "Resume-markdown-docling"
MAX_RESUMES = 200

In [6]:
# Basic PDF to Markdown conversion
from pathlib import Path

# Setup paths
pdf_dir = Path("Submit your resume or CV (File responses)")
output_dir = Path("Resume-markdown-docling")
output_dir.mkdir(exist_ok=True)

# Get first 200 PDFs
pdf_files = list(pdf_dir.glob("*.pdf"))[:200]
print(f"Found {len(pdf_files)} PDFs to convert")

# Convert PDFs
converter = DocumentConverter()
successful = 0

for pdf_file in pdf_files:
    try:
        result = converter.convert(str(pdf_file))
        markdown_content = result.document.export_to_markdown()

        output_file = output_dir / f"{pdf_file.stem}.md"
        output_file.write_text(markdown_content, encoding='utf-8')

        successful += 1
        if successful % 20 == 0:
            print(f"Converted {successful} files")

    except Exception as e:
        print(f"Failed: {pdf_file.name}")

print(f"Conversion complete: {successful}/{len(pdf_files)} successful")

Found 52 PDFs to convert


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Converted 20 files
Converted 40 files
Conversion complete: 52/52 successful


In [7]:


markdown_dir=Path("Resume-markdown-docling")
def fix_spaced_caps(s: str) -> str:
    pattern = re.compile(r'(?<!\w)(?:[A-Z]\s+){2,}[A-Z](?!\w)')  # 3+ capital letters separated by spaces
    def _join(m):
        return m.group(0).replace(' ', '')
    return pattern.sub(_join, s)

def container_chunking(content: str, resume_id: str) -> List[Document]:
    # Remove image tags
    content = re.sub(r'<!-- image -->', '', content)
    #Removing spacing S K I L L -> SKILL
    # Fix spaced out words like "e x p e r i e n c e" or "E X P E R I E N C E"
    # content = re.sub(r'\b(\w)\s+(\w)\s+(\w)(\s+\w)*\b', lambda m: re.sub(r'\s+', '', m.group()), content)
    content = fix_spaced_caps(content)

    containers = [
        r'about\s*me', r'summary', r'profile', r'experience', r'work\s+experience',
        r'education', r'skill[s]?', r'project[s]?', r'achievement[s]?', r'award[s]?',
        r'publication[s]?', r'competition[s]?', r'hackathon[s]?'
    ]
    container_alt = '|'.join(containers)

    # Anchor to line start, any H1–H6, match only the heading line
    pattern = rf'(?=^#{{1,6}}\s*(?:.*\b(?:{container_alt})\b).*$)'
    chunks = re.split(pattern, content, flags=re.IGNORECASE | re.MULTILINE)

    # Filter empty chunks and create documents
    docs = []
    for i, chunk in enumerate(chunks):
        chunk = chunk.strip()
        if chunk and len(chunk) > 50:
            doc = Document(
                page_content=chunk,
                metadata={'resume_id': resume_id, 'chunk_id': i}
            )
            docs.append(doc)

    return docs

# Process all resumes
all_chunks = []
for md_file in markdown_dir.glob("*.md"):
    content = md_file.read_text(encoding='utf-8')
    resume_id = md_file.stem
    chunks = container_chunking(content, resume_id)
    all_chunks.extend(chunks)

print(f"Created {len(all_chunks)} chunks from {len(list(markdown_dir.glob('*.md')))} resumes")
print(f"Average chunks per resume: {len(all_chunks) / len(list(markdown_dir.glob('*.md'))):.1f}")

Created 250 chunks from 52 resumes
Average chunks per resume: 4.8


Embedding Models

In [4]:

# Store API key in Colab secrets, then access it
openai.api_key = userdata.get('OPENAI_API_KEY')

In [9]:


# use existing openai.api_key if set, else read from env
api_key = getattr(openai, "api_key", None) or os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OpenAI API key not found. Set openai.api_key or OPENAI_API_KEY env var.")
client = OpenAI(api_key=api_key)

def get_openai_embeddings(texts: list[str], model: str = "text-embedding-3-small", batch_size: int = 100) -> np.ndarray:
    embeddings: list[np.ndarray] = []
    for i in tqdm(range(0, len(texts), batch_size), desc="Embedding batches"):
        batch = texts[i : i + batch_size]
        resp = client.embeddings.create(model=model, input=batch)
        embeddings.extend([np.array(item.embedding, dtype=np.float32) for item in resp.data])
    return np.vstack(embeddings) if embeddings else np.zeros((0, 0), dtype=np.float32)

# prepare chunks (prefer in-memory all_chunks, else load saved chunks)
try:
    chunks = all_chunks  # type: ignore[name-defined]
except NameError:
    if os.path.exists("resume_chunks.pkl"):
        with open("resume_chunks.pkl", "rb") as f:
            chunks = pickle.load(f)
    else:
        raise RuntimeError("No in-memory chunks and resume_chunks.pkl not found.")

chunk_texts = [doc.page_content for doc in chunks]
print(f"Generating OpenAI embeddings for {len(chunk_texts)} chunks...")

# choose "text-embedding-3-small" or "text-embedding-3-large"
embeddings = get_openai_embeddings(chunk_texts, model="text-embedding-3-small", batch_size=100)

# save chunks and embeddings
with open("resume_chunks_openai.pkl", "wb") as f:
    pickle.dump(chunks, f)

with open("resume_embeddings_openai.pkl", "wb") as f:
    pickle.dump(embeddings, f)

print(f"Saved {len(chunks)} chunks and embeddings -> resume_chunks_openai.pkl, resume_embeddings_openai.pkl")
print(f"Embeddings shape: {embeddings.shape}")

# retrieval function using cosine similarity
def retrieve_similar_chunks_openai(query: str, top_k: int = 5, model: str = "text-embedding-3-small"):
    # ensure embeddings & chunks are loaded in scope
    global embeddings, chunks
    if 'embeddings' not in globals() or embeddings is None or embeddings.size == 0:
        if os.path.exists("resume_embeddings_openai.pkl"):
            with open("resume_embeddings_openai.pkl", "rb") as f:
                embeddings = pickle.load(f)
        else:
            raise RuntimeError("Embeddings not loaded. Run embedding generation first.")
    if 'chunks' not in globals() or chunks is None:
        if os.path.exists("resume_chunks_openai.pkl"):
            with open("resume_chunks_openai.pkl", "rb") as f:
                chunks = pickle.load(f)
        else:
            raise RuntimeError("Chunks not loaded. Run embedding generation first.")


    0# get query embedding
    q_resp = client.embeddings.create(model=model, input=[query])
    q_emb = np.array(q_resp.data[0].embedding, dtype=np.float32).reshape(1, -1)
    sims = cosine_similarity(q_emb, embeddings)[0]
    top_idx = np.argsort(sims)[-top_k:][::-1]
    results = []
    for i in top_idx:
        results.append({
            "score": float(sims[i]),
            "content": chunks[i].page_content,
            "metadata": chunks[i].metadata
        })
    return results



Generating OpenAI embeddings for 250 chunks...


Embedding batches: 100%|██████████| 3/3 [00:05<00:00,  1.71s/it]

Saved 250 chunks and embeddings -> resume_chunks_openai.pkl, resume_embeddings_openai.pkl
Embeddings shape: (250, 1536)


In [10]:
# example usage:
results = retrieve_similar_chunks_openai("student who has scored more than 95% in school", top_k=3)
# for r in results: print(r["score"], r["metadata"]["resume_id"], r["content"][:200])

In [11]:
results

[{'score': 0.5160472393035889,
  'content': '## EDUCATION\n\n|   Year | Degree/Exam                 | Institute                                  | CGPA/Marks   |\n|--------|-----------------------------|--------------------------------------------|--------------|\n|   2026 | Dual Degree (B.Tech+M.Tech) | IIT Kharagpur                              | 8.80/ 10     |\n|   2021 | CLASS XII                   | Christ Church Boys Senior Secondary School | 95.60%       |\n|   2019 | CLASS X                     | Christ Church Boys Senior Secondary School | 96.40%       |',
  'metadata': {'resume_id': 'zaid_resume_data - Zaid Ahmed Khan',
   'chunk_id': 1}},
 {'score': 0.47852623462677,
  'content': '## EDUCATION\n\nD G VAISHNAV COLLEGE [2017-2020]\n\nBachelor of Science in Physics with Computer Applications OOPS, Basics of C++, JAVA, Data structure Marks obtained - 71%\n\nICF SILVER JUBILEE MATRICULATION HIGHER SECONDARY SCHOOL HSC - STATE BOARD [2016-2017] BIOLOGY / MATHS Marks obtained - 84%

Trying Hybrid Retrieval

In [9]:
# Load saved chunks and embeddings
with open('resume_chunks_openai.pkl', 'rb') as f:
    chunks = pickle.load(f)

with open('resume_embeddings_openai.pkl', 'rb') as f:
    embeddings = pickle.load(f)

# --- tiny cleaner & tokenizer ---
_CLEAN_RX = re.compile(r'[^\w\s]')
_WS_RX = re.compile(r'\s+')

def clean_and_tokenize(text: str) -> List[str]:
    text = text.lower()
    text = _CLEAN_RX.sub(' ', text)     # remove punctuation
    text = _WS_RX.sub(' ', text).strip()
    return [t for t in text.split(' ') if t]

class BM25Index:
    def __init__(self):
        self.bm25 = None
        self.docs: List[Document] = []
        self.doc_tokens: List[List[str]] = []

    def fit(self, chunks: List[Document]):
        self.docs = chunks
        self.doc_tokens = [clean_and_tokenize(d.page_content) for d in chunks]
        self.bm25 = BM25Okapi(self.doc_tokens)

    def search(self, query: str, top_k: int = 200) -> List[Dict[str, Any]]:
        if self.bm25 is None:
            raise RuntimeError("Call fit(chunks) before search().")
        q_tokens = clean_and_tokenize(query)
        scores = self.bm25.get_scores(q_tokens)
        # top-k indices by score
        top_idx = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
        out = []
        for rank, i in enumerate(top_idx, 1):
            d = self.docs[i]
            out.append({
                "rank": rank,
                "bm25_score": float(scores[i]),
                "resume_id": (d.metadata or {}).get("resume_id"),
                "chunk_id": (d.metadata or {}).get("chunk_id"),
                "preview": d.page_content[:400]
            })
        return out


In [10]:
bm25 = BM25Index()
bm25.fit(chunks)  # chunks = your List[Document] from the chunker

query = "Mumbai University"
hits = bm25.search(query, top_k=300)

print(f"BM25 hits: {len(hits)}")
for h in hits[:5]:
    print(f"[{h['rank']:>2}] {h['bm25_score']:.3f}  resume={h['resume_id']}  chunk={h['chunk_id']}")
    print("   ", (h["preview"] or "").splitlines()[0][:540], "...")


BM25 hits: 249
[ 1] 6.464  resume=SHAHNAWAZ_RESUME (4) - shahnawaz Shaikh  chunk=2
    ## Education ...
[ 2] 5.472  resume=(Shivam Dubey)-Resume - Shivam Shailendra Dubey  chunk=1
    ## PROFILE SUMMARY ...
[ 3] 5.099  resume=VES_Ganesh_Deulkar_Resume - Ganesh Deulkar  chunk=1
    ## Education ...
[ 4] 4.662  resume=VES_Ganesh_Deulkar_Resume - Ganesh Deulkar  chunk=4
    ## Technical Skills ...
[ 5] 4.419  resume=SHAHNAWAZ_RESUME (4) - shahnawaz Shaikh  chunk=3
    ## Experience ...


In [11]:
import numpy as np
from typing import List, Dict, Any
from langchain.docstore.document import Document
from openai import OpenAI


api_key = getattr(openai, "api_key", None) or os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OpenAI API key not found. Set openai.api_key or OPENAI_API_KEY env var.")
client = OpenAI(api_key=api_key)

class DenseIndexSimple:
    def __init__(self):
        self.docs: List[Document] = []
        self.Xn: np.ndarray | None = None  # L2-normalized embeddings (N, D)
        self.dim: int | None = None
        self.model_name: str | None = None

    def fit(self, chunks: List[Document], embeddings: np.ndarray, model_name: str):
        """
        chunks: your List[Document]
        embeddings: np.ndarray of shape (N, D) aligned with chunks
        model_name: the embedding model used (e.g., 'text-embedding-3-small')
        """
        if embeddings.ndim != 2 or len(chunks) != embeddings.shape[0]:
            raise ValueError("Embeddings must be 2D and aligned with chunks.")
        X = embeddings.astype(np.float32)
        X = np.ascontiguousarray(X)
        norms = np.linalg.norm(X, axis=1, keepdims=True) + 1e-12
        self.Xn = X / norms
        self.dim = self.Xn.shape[1]
        self.docs = chunks
        self.model_name = model_name

    def _embed_query(self, query: str) -> np.ndarray:
        if not self.model_name:
            raise RuntimeError("Index not initialized with a model_name. Call fit() first.")
        resp = client.embeddings.create(model=self.model_name, input=[query])
        q = np.array(resp.data[0].embedding, dtype=np.float32)
        if q.shape[0] != self.dim:
            raise ValueError(f"Query dim {q.shape[0]} != index dim {self.dim}. "
                             f"Use the same embedding model as indexing.")
        q = q / (np.linalg.norm(q) + 1e-12)
        return q  # (D,)

    def search(self, query: str, top_k: int = 200) -> list[dict]:
        if self.Xn is None:
            raise RuntimeError("Index empty. Call fit() first.")

        # embed + normalize query
        q = self._embed_query(query)                 # (D,)
        sims = self.Xn @ q                           # (N,)
        n = sims.shape[0]
        k = max(1, min(top_k, n))                    # clamp to [1, N]

        # For small N, argsort is fine and simpler
        top_idx = np.argsort(sims)[::-1][:k]

        results = []
        for rank, i in enumerate(top_idx, 1):
            d = self.docs[i]
            results.append({
                "rank": rank,
                "dense_score": float(sims[i]),
                "resume_id": (d.metadata or {}).get("resume_id"),
                "chunk_id": (d.metadata or {}).get("chunk_id"),
                "preview": d.page_content[:400],
            })
        return results



In [12]:
# You already have:
# - chunks: List[Document]
# - embeddings: np.ndarray from your earlier OpenAI embedding step
# - model_name: e.g., "text-embedding-3-small" (must match the one used for embeddings)

dense = DenseIndexSimple()
dense.fit(chunks, embeddings, model_name="text-embedding-3-small")

query = "mumbai university"
dense_hits = dense.search(query, top_k=300)

print(f"Dense hits: {len(dense_hits)}")
for h in dense_hits[:5]:
    print(f"[{h['rank']:>2}] {h['dense_score']:.4f} resume={h['resume_id']} chunk={h['chunk_id']}")


Dense hits: 249
[ 1] 0.4334 resume=VES_Ganesh_Deulkar_Resume - Ganesh Deulkar chunk=1
[ 2] 0.3864 resume=SHAHNAWAZ_RESUME (4) - shahnawaz Shaikh chunk=2
[ 3] 0.3802 resume=Saaquib Motiwala Resume-6 (1) - saaquib motiwala chunk=3
[ 4] 0.3674 resume=Mohit_CV - Mohit Lohani chunk=1
[ 5] 0.3660 resume=Resume Mollika - Mollika Garg chunk=1


In [15]:
import hashlib
from typing import List, Dict, Any, Optional

def _make_key(hit: Dict[str, Any]) -> str:
    """Stable key to identify a chunk across lists."""
    rid = (hit.get("resume_id") or "").strip()
    cid = str(hit.get("chunk_id") or "").strip()
    if rid or cid:
        return f"{rid}::{cid}"
    # Fallback: hash preview if metadata missing
    prev = (hit.get("preview") or "")[:256]
    return "hash::" + hashlib.md5(prev.encode("utf-8")).hexdigest()

def rrf_fuse(
    bm25_hits: Optional[List[Dict[str, Any]]],
    dense_hits: Optional[List[Dict[str, Any]]],
    k: int = 60,
    top_k: int = 100,
    weights: Dict[str, float] = None,
) -> List[Dict[str, Any]]:
    """
    Reciprocal Rank Fusion (RRF): score = sum_s w_s * 1/(k + rank_s)
    - bm25_hits / dense_hits: lists with at least {'rank', 'resume_id', 'chunk_id'} (or 'preview').
    - k: stabilization constant (common choices: 60, 100).
    - top_k: number of fused results to return.
    - weights: optional per-source weights, e.g., {'bm25': 1.0, 'dense': 1.0}
    """
    weights = weights or {"bm25": 1.0, "dense": 1.0}
    pool: Dict[str, Dict[str, Any]] = {}

    def add_source(hits: Optional[List[Dict[str, Any]]], label: str):
        if not hits:
            return
        for h in hits:
            key = _make_key(h)
            rec = pool.setdefault(key, {
                "resume_id": h.get("resume_id"),
                "chunk_id": h.get("chunk_id"),
                "preview": h.get("preview"),
                # keep original per-source info if present
                "bm25_rank": None, "bm25_score": None,
                "dense_rank": None, "dense_score": None,
                "rrf_score": 0.0,
            })
            r = h.get("rank")
            if isinstance(r, int) and r >= 1:
                rec["rrf_score"] += weights.get(label, 1.0) * (1.0 / (k + r))
            # stash per-source details (first occurrence wins)
            rank_key = f"{label}_rank"
            score_key = f"{label}_score"
            if rec[rank_key] is None:
                rec[rank_key] = r
            if rec[score_key] is None:
                # hit may have 'bm25_score' or 'dense_score'
                val = h.get(score_key) or h.get("bm25_score") or h.get("dense_score")
                rec[score_key] = float(val) if val is not None else None

    add_source(bm25_hits, "bm25")
    add_source(dense_hits, "dense")

    fused = sorted(pool.values(), key=lambda x: x["rrf_score"], reverse=True)[:top_k]
    # add final rank
    for i, rec in enumerate(fused, 1):
        rec["rank"] = i
    return fused


In [16]:
query="A graduate from Mumbai University"
bm25_hits = bm25.search(query, top_k=500)
dense_hits = dense.search(query, top_k=300)

fused = rrf_fuse(bm25_hits, dense_hits, k=60, top_k=100)
for r in fused[:10]:
    print(f"[{r['rank']:>2}] RRF={r['rrf_score']:.5f}  bm25_r={r['bm25_rank']}  dense_r={r['dense_rank']}")
    print("   ", (r["preview"] or "").splitlines()[0][:120], "...")


[ 1] RRF=0.03227  bm25_r=1  dense_r=3
    ## PROFILE SUMMARY ...
[ 2] RRF=0.03200  bm25_r=3  dense_r=2
    ## Education ...
[ 3] RRF=0.03175  bm25_r=2  dense_r=4
    ## Education ...
[ 4] RRF=0.03089  bm25_r=9  dense_r=1
    ## EDUCATION ...
[ 5] RRF=0.02797  bm25_r=11  dense_r=12
    ## EDUCATIOn ...
[ 6] RRF=0.02722  bm25_r=12  dense_r=15
    ## Anshuman Awasthi MTech(Computer Science) Indian Institute of Information Technology Lucknow ...
[ 7] RRF=0.02699  bm25_r=4  dense_r=28
    ## Technical Skills ...
[ 8] RRF=0.02586  bm25_r=28  dense_r=9
    ## Education ...
[ 9] RRF=0.02579  bm25_r=34  dense_r=6
    ## Education: ...
[10] RRF=0.02558  bm25_r=27  dense_r=11
    ## Education ...


In [17]:
for r in fused[:10]:
    print(r.get("resume_id", ""))

(Shivam Dubey)-Resume - Shivam Shailendra Dubey
VES_Ganesh_Deulkar_Resume - Ganesh Deulkar
SHAHNAWAZ_RESUME (4) - shahnawaz Shaikh
Saaquib Motiwala Resume-6 (1) - saaquib motiwala
Mohit_CV - Mohit Lohani
Anshuman_Resume_Final 3 - ANSHUMAN AWASTHI
VES_Ganesh_Deulkar_Resume - Ganesh Deulkar
Resume_Shikhar_USAR(G.G.S.I.P.U) - Shikhar Kanojia
Asmita  Bele resume - Asmita Bele
resume (2) - Diksha Uniyal
